In [1]:
import math
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from scipy.optimize import fsolve
from scipy.integrate import quad
from fun_utils_eq import *
#  The function "equilibrium_hatalgebra" contained equilibrium conditions, which allows us to calculate the equilibrium price with given set of taxes
#  The function "minuswelfare" contained equations to calculate Home's welfare.
    ## The only difference between the two minuswelfare function is the fisrt argument input: I do this because scipy.minimize cosiders the first argument as the unkown to be solved by default.
#  The function "minimization" is used to maximize Home's welfare to get optimal tax values and energy price.
#  The function "callback_pe" takes the tax values calculated by welfare maximization to back out corresponding energy price.
#  The function "callback" takes optimal taxes and energy price to recalculate all other values including world emissions, home emissions...

In [2]:
import math
import numpy as np
import pandas as pd
from scipy.optimize import minimize, least_squares
from scipy.optimize import fsolve
from scipy.integrate import quad
import matplotlib.pyplot as plt

In [12]:
## parameter values
alpha = 0.85           # labor share parameter in manufacturing
theta = 2             # scopevec for comparative advantage
sigma = 1      # elasticity of demand for each individual manufactured good j at Home
sigmastar = 1  # elasticity of demand for each individual manufactured good j at Foreign
epsilonD = alpha + (1 - alpha) * sigma  #Home's elasticity of demand for embodied energy
epsilonDstar = alpha + (1 - alpha) * sigmastar  #Foreign's elasticity of demand for embodied energy
# beta = 2.274853
# gamma= 0.784877595
beta=1.892412
gamma=0.807998928
epsilonS = 0.5  #Homes's energy supply elasticity: beta/(1 - beta)
epsilonSstar = 0.5  #Foreign's energy supply elasticity: betastar/(1 - betastar)

In [13]:
## scenario switch
case=3 # 2 means no trade in goods; 3 means trade in both energy and goods
logit = 0 # 1 means logit estimations of supply elasticity; 0 means fixed elasticities at 0.5

In [14]:
ParaList = (alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit)

In [21]:
class taxrow_eq:
    def __init__(self, pe, te, tb_mat, df, tax_scenario, varphi, ParaList):
        self.df = df
        self.tax_scenario = tax_scenario
        self.varphi = varphi
        self.ParaList = ParaList
        self.prop = 1
        self.te = te
        self.pe = pe
        self.tb_mat = tb_mat

    ## computes equilibrium price and taxes. Also computes other equilibrium values (ie consumption, production, value of exports/imports)
    ## and stores them in self. 
    def opt_tax(self):
        tax_scenario = self.tax_scenario
        varphi = self.varphi
        df = self.df
        te = varphi
        self.region = df['region_scenario']
        pes = np.append([self.pe], np.arange(0.1, 2.5, 0.2))
        tbs = np.append([self.tb_mat[0]], np.arange(0,1.5,0.3))

        props = np.append([self.tb_mat[1]], np.arange(0,1.1,0.5))
        self.conv = 1
        #pes = [self.pe]
        #tes = [self.te]
        
        if tax_scenario['tax_sce'] != 'purete' and tax_scenario['tax_sce'] != 'EP_hybrid':
            for prop in props:
                for tb in tbs:
                    for pe in pes:
                        res = minimize(self.minuswelfare, [pe, tb, prop], bounds=[(0.01, np.inf), (0, np.inf), (0, 1)],
                               method='nelder-mead', args=(te), tol=0.00001, options={'maxfev': 100000})
                        if res.fun <= 0.0001:
                            opt_val = res.x
                            break
                    else:
                        continue
                    break
                else: 
                    continue
                break
            if res.fun > 0.0001:
                self.conv = 0
                print("did not converge, phi is", varphi, "init guess is", self.pe, self.tb_mat, 'region is', tax_scenario['tax_sce'], df['region_scenario'])
            opt_val = res.x
            #print(res.fun, df['region_scenario'], tax_scenario['tax_sce'], varphi)
            
        pes = np.append([self.pe], np.arange(0.1, 2, 0.5))
        props = np.append([self.tb_mat[1]], np.arange(0,2,0.5))
        if tax_scenario['tax_sce'] == 'EP_hybrid':
            for pe in pes:
                for tb in tbs:
                    for prop in props:
                        res = minimize(self.minuswelfare, [pe, tb, prop], bounds=[(0.01, np.inf), (0, np.inf), (0, np.inf)],
                               method='nelder-mead', args=(te), tol=0.00001, options={'maxfev': 100000})
                        if res.fun <= 0.0001:
                            opt_val = res.x
                            break
                    else:
                        continue
                    break
                else: 
                    continue
                break
            if res.fun > 0.0001:
                self.conv = 0
                print("did not converge, phi is", varphi, "init guess is", self.pe, self.tb_mat, 'region is', tax_scenario['tax_sce'], df['region_scenario'])
            opt_val = res.x
            
            
        ## not implemented yet
        if tax_scenario['tax_sce'] == 'Baseline':
            self.tb = 0
            self.prop = 1
            self.te = 0

        elif tax_scenario['tax_sce'] == 'PC_hybrid':
            self.tb = opt_val[1]
            self.prop = opt_val[2]
            self.te = self.tb

        elif tax_scenario['tax_sce'] == 'EPC_hybrid':
            self.tb = opt_val[1]
            self.prop = opt_val[2]
            self.te = varphi

        elif tax_scenario['tax_sce'] == 'EP_hybrid':
            self.tb = opt_val[1]
            self.te = opt_val[2]
            self.prop = self.te #- self.tb

        elif tax_scenario['tax_sce'] == 'Unilateral' or tax_scenario['tax_sce'] == 'puretc' or tax_scenario[
            'tax_sce'] == 'puretp' or tax_scenario['tax_sce'] == 'EC_hybrid':
            self.tb = opt_val[1]
            self.prop = opt_val[2]

            if tax_scenario['tax_sce'] == 'puretc' or tax_scenario['tax_sce'] == 'puretp':
                self.te = self.tb

            elif tax_scenario['tax_sce'] == 'Unilateral' or tax_scenario['tax_sce'] == 'EC_hybrid':
                self.te = self.varphi

        elif tax_scenario['tax_sce'] == 'purete':
            res = minimize(self.minuswelfare, [self.pe, self.te], bounds=[(0, np.inf), (0, np.inf)], method='nelder-mead', args=(0),
                           tol=0.000001, options={'maxfev': 100000})
            
            if res.fun > 0.0001:
                self.conv = 0
                print("did not converge, phi is", varphi, "init guess is", self.pe, self.tb_mat, 'region is', tax_scenario['tax_sce'], df['region_scenario'])
            #print(res.fun)
            self.te = res.x[1]
            tb_mat = [0, 1]
            self.tb = tb_mat[0]
            self.prop = tb_mat[1]
        #print(tax_scenario['tax_sce'], res)

    def minuswelfare(self, p, te):
        pe = p[0]
        tb_mat = p[1:]
        varphi = self.varphi
        tax_scenario = self.tax_scenario
        ParaList = self.ParaList
        df = self.df

        if tax_scenario['tax_sce'] == 'purete':
            te = p[1]
            tb_mat = [0, 1]

        obj = self.comp_obj(pe, te, tb_mat, varphi, tax_scenario, ParaList, df)

        return obj

    ## compute the objective value, currently the objective is to minimize negative welfare
    ## also saves optimal results in self.
    def comp_obj(self, pe, te, tb_mat, varphi, tax_scenario, Paralist, df):

        alpha, theta, sigma, sigmastar, epsilonD, epsilonDstar, epsilonS, epsilonSstar, beta, gamma, logit = ParaList
        ## compute extraction tax, and jbar's
        te, jxbar_hat, jmbar_hat, j0_hat, tb_mat = computejbar(ParaList, pe, te, varphi, tb_mat, tax_scenario, df)

        jxbar_prime = jxbar_hat * df['jxbar']
        jmbar_prime = jmbar_hat * df['jmbar']
        j0_prime = j0_hat * df['jxbar']

        jvals = (j0_hat, j0_prime, jxbar_hat, jxbar_prime, jmbar_hat, jmbar_prime)

        # if te is too large, Home stop producing
        petbte = pe + tb_mat[0] - te
        z = pe + tb_mat[0] >= te
        petbte = petbte * z

        Qe_prime, Qestar_prime = compute_qe(petbte, epsilonS, epsilonSstar, logit, beta, gamma, pe, df)

        CeHH_prime, CeFH1_prime, CeFH2_prime, CeFH_prime, CeHF_prime, CeFF_prime, CeFH_hat, CeFH1_hat, CeHF_hat = comp_ce(
            pe, tb_mat, jvals, ParaList, df, tax_scenario)

        Ge_prime = CeHH_prime + CeFH_prime
        Gestar_prime = CeFF_prime + CeHF_prime
        Ce_prime = CeHH_prime + CeHF_prime
        Cestar_prime = CeFF_prime + CeFH_prime
        Qeworld_prime = Qe_prime + Qestar_prime

        consvals = (CeHH_prime, CeFH1_prime, CeFH2_prime, CeFH_prime, 
                    CeHF_prime, CeFF_prime, CeFH_hat, CeFH1_hat, CeHF_hat,
                    Ce_prime, Cestar_prime)

        VgHH, VgFF, VgFH1_prime, VgFH2_prime, VgFH_prime, VgHF_prime, VgFH, VgHF = comp_vg(pe, tb_mat, jvals, consvals,
                                                                                           df, tax_scenario, ParaList)

        pai_g = VgFH - (pe + tb_mat[0]) * df['CeFH'] / (1 - alpha)
        subsidy_ratio = (tb_mat[0] / pe * (1 - alpha)) / (1 + tb_mat[0] / pe * (1 - alpha))

        ## compute Ve values
        Ve_prime, Vestar_prime = comp_ve(pe, tb_mat, consvals, tax_scenario)

        ## compute more Vg values
        Vg, Vg_prime, Vgstar, Vgstar_prime = comp_vgfin(pe, tb_mat, consvals, VgFH_prime, ParaList, df, tax_scenario)

        Lg, Lg_prime, Lgstar, Lgstar_prime = comp_lg(pe, tb_mat, Ge_prime, Gestar_prime, consvals, ParaList, df, tax_scenario)

        leakage1, leakage2, leakage3 = comp_leak(Qestar_prime, Gestar_prime, Cestar_prime, Qeworld_prime, df)

        delta_Le, delta_Lestar, delta_U = comp_delta(Lg, Lg_prime, Lgstar, Lgstar_prime, Qeworld_prime, Vg, Vgstar,
                                                         df, jvals, pe, petbte, tb_mat, tax_scenario, varphi, ParaList)

        chg_extraction, chg_production, chg_consumption, chg_Qeworld = comp_chg(df, Qestar_prime, Gestar_prime,
                                                                                Cestar_prime, Qeworld_prime)

        welfare = delta_U / Vg * 100
        welfare_noexternality = (delta_U + varphi * (Qeworld_prime - df['Qeworld'])) / Vg * 100

        self.results = assign_val(Ge_prime, Gestar_prime, Lg_prime, Lgstar_prime, Qe_prime, Qestar_prime, Qeworld_prime,
                                  Ve_prime, Vestar_prime, VgFH1_prime, VgFH2_prime, VgFH_prime, VgHF_prime, Vg_prime,
                                  Vgstar_prime, chg_Qeworld, chg_consumption, chg_extraction, chg_production, delta_Le, 
                                  delta_Lestar, leakage1, leakage2, leakage3,
                                  pai_g, pe, subsidy_ratio, varphi, welfare, welfare_noexternality, jvals, consvals)

        diff, diff1, diff2 = comp_diff(consvals, jvals, Ge_prime, Gestar_prime, Qe_prime, Qestar_prime, Qeworld_prime,
                                       VgFH2_prime, pe, tax_scenario, tb_mat, te, varphi, ParaList, df)

        return abs(diff1) + abs(diff) + abs(diff2)

    ## retrive the pandas series object containing equilibrium values (prices, taxes, consumption etc)
    def retrive(self):
        ret = self.results
        ret['tb'] = self.tb
        ret['prop'] = self.prop
        ret['te'] = self.te
        ret['region_scenario'] = self.region
        ret['conv'] = self.conv
        
        return ret

In [22]:
## import BAU values (seven regional scenarios in the order of US, EU, OECD, World, China, OECD plus China)
if case==2:
    df = pd.read_csv("../output/BaselineCarbon_2015_noTradeinGoods.csv",index_col=['region_scenario','regionbase'],header='infer')
elif case==3:
    df = pd.read_csv("../output/BaselineCarbon_2015.csv", index_col=['regionbase'],header='infer')
df['jxbar']=df['CeFH']/(df['CeFH'] + df['CeFF'])
df['jmbar']=df['CeHH']/(df['CeHH'] + df['CeHF'])

## choose which regional scenario to run (runs all if not executed)
#df=df.drop([2,3,4,5,6,7])  
df = df.drop(['World as Home'])
df

,region_scenario,Qe,Qestar,Qeworld,CeHH,CeHF,CeFH,CeFF,Ce,Cestar,Ge,Gestar,Ceworld,Geworld,jxbar,jmbar
regionbase,,,,,,,,,,,,,,,,
US as Home,1,4.480045,27.795946,32.275990,4.598403,1.196111,0.421601,26.059874,5.794514,26.481474,5.020004,27.255985,32.275990,32.275990,0.015921,0.793579
EU28 as Home,2,0.935781,31.340202,32.275986,2.950581,1.013552,0.507676,27.804173,3.964133,28.311848,3.458257,28.817724,32.275982,32.275982,0.017932,0.744319
OECD37 as Home,3,8.625495,23.650503,32.276001,11.293668,2.487537,0.910579,17.584217,13.781205,18.494795,12.204247,20.071754,32.276001,32.276001,0.049234,0.819498
China as Home,5,7.522744,24.753246,32.275990,7.345464,0.632472,1.935382,22.362669,7.977936,24.298052,9.280846,22.995142,32.275990,32.275990,0.079652,0.920722
OECD and China as Home,6,16.148245,16.127758,32.276001,20.103180,1.655961,1.381915,9.134943,21.759140,10.516858,21.485096,10.790904,32.276001,32.276001,0.131400,0.923896
US and EU as Home,7,5.415825,26.860159,32.275986,7.728876,2.029770,0.749387,21.767950,9.758646,22.517336,8.478263,23.797718,32.275982,32.275982,0.033280,0.792003


In [23]:
tax_scenario= pd.DataFrame({'tax_sce': ['Unilateral','purete','puretc','puretp','EC_hybrid','EP_hybrid','PC_hybrid','EPC_hybrid'], 'Base':[0,1,1,1,1,1,1,1]},index=[2, 3, 4, 5, 6, 7, 8, 9])
#tax_scenario= pd.DataFrame({'tax_sce': ['EP_hybrid','EPC_hybrid'], 'Base':[1,1]},index=[3, 4])
#df = pd.DataFrame({'Qe': 4.4800448, 'Qestar': 27.795946, 'Qeworld': 32.27599, 'CeHH': 4.598403, 'CeHF': 1.196111,'CeFH': 0.42160103,'CeFF': 26.059874,'Ce': 5.7945137,'Cestar': 26.481474,'Ge': 5.0200038,'Gestar': 27.255985,'Ceworld': 32.27599,'Geworld': 32.27599,'jxbar': 0.015920602214279298,'jmbar':  0.7935787194577492}, index = [1])

In [24]:
def temp(tax_scenario, alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit):
    ParaList = (alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit)
    te = 0; #initial value of extraction tax for iteration
    tb_mat = [0, 1];  #initial value of  border adjustment and proportion of it (prop is mainly used for PC hybrid)
    # use for quick test: varphilist = [2] or varphilist = np.arange (1.7,2.5,0.1)
    varphilist = [2]
    varphilist = np.arange (0,20,0.1)
    varphilist = np.append(varphilist, np.arange(20,50,1))
    output=[]
    prevtb = [1,1,1, 1]
    for varphi in varphilist:    
        te = 0 #initial value of extraction tax for iteration
        tb_mat = [0, 1]  #initial value of  border adjustment and proportion of it (prop is mainly used for PC hybrid)
        ## calculate for optimal taxes by maximizing welfare
        tax_df=df.apply(mintemp, axis=1, raw=False, args=(prevtb, tb_mat, te, varphi, tax_scenario, ParaList))
        prevtb = tax_df[['region_scenario','tb', 'pe', 'te', 'prop']]
        #print(prevtb)
        output.append(tax_df)
        #print(varphi)
    output = pd.concat(output, axis=0, join='outer',  ignore_index=False, keys=None, levels=None, names=None, verify_integrity=False,copy=True)
    output.reset_index(level=0, inplace=True)
    output = output.sort_values(by=['region_scenario','varphi'])
    if tax_scenario['tax_sce']=='purete' or tax_scenario['tax_sce']=='EP_hybrid':
        output.te[output.Qe_prime==0]=output.pe+output.tb
    print(tax_scenario['tax_sce'])
    return output

In [25]:
def mintemp(df, prevtb, tb_mat, te, varphi, tax_scenario, ParaList):
    pe = 0.5
    te = 0.5
    tb_mat = [0.5,0.5]
    #print(prevtb['region_scenario'])
    
    if (type(prevtb) != list):
        curr_region = prevtb[prevtb['region_scenario'] == df['region_scenario']]
        vals = curr_region.values
        #print(vals)
        tb_mat = [vals[0][1], vals[0][4]]
        pe = vals[0][2]
        te = vals[0][3]
    tax_temp = taxrow_eq(pe, te, tb_mat, df, tax_scenario, varphi, ParaList)
    tax_temp.opt_tax()
    ret = tax_temp.retrive()
    #print(ret)
    return ret

In [26]:
temp_df = tax_scenario.apply(temp, axis=1, args=(alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit))
#temp_df.head()

Unilateral
purete
puretc
puretp
EC_hybrid
EP_hybrid
did not converge, phi is 0.1 init guess is 0.9999996674149161 [0.0, 0.7707859372524186] region is PC_hybrid 1.0
did not converge, phi is 0.1 init guess is 0.9999996674149161 [0.0, 0.7707859372524186] region is PC_hybrid 7.0
PC_hybrid
EPC_hybrid


In [27]:
output_list=[]
for i in range(2,len(tax_scenario)+2):
    output_list.append(temp_df.loc[i])
Outcomes = pd.concat(output_list, axis=0, join='outer', ignore_index=False, keys=tax_scenario['tax_sce'], levels=None, verify_integrity=False,copy=True)
Outcomes.reset_index(level=0, inplace=True)
Outcomes

,tax_sce,regionbase,varphi,pe,tb,prop,te,jxbar_prime,jmbar_prime,j0_prime,...,chg_extraction,chg_production,chg_consumption,chg_Qeworld,pai_g,subsidy_ratio,welfare,welfare_noexternality,region_scenario,conv
0,Unilateral,US as Home,0.0,1.000000,0.000000,0.411799,0.0,0.015921,0.793579,0.015921,...,1.666413e-07,-3.268083e-07,7.124784e-07,9.934999e-07,-3.370090e-08,0.000000,-0.000005,-0.000005,1.0,1.0
6,Unilateral,US as Home,0.1,0.998752,0.034739,0.000000,0.1,0.015923,0.793579,0.015761,...,-1.734509e-02,-6.269528e-03,1.890139e-02,-1.688885e-01,-9.413296e-02,0.005190,0.021913,-0.021806,1.0,1.0
12,Unilateral,US as Home,0.2,0.997896,0.069419,0.000091,0.2,0.015930,0.793579,0.015608,...,-2.926299e-02,-2.072373e-02,2.839438e-02,-3.370540e-01,-1.891988e-01,0.010327,0.087356,-0.087148,1.0,1.0
18,Unilateral,US as Home,0.3,0.997415,0.104080,0.000055,0.3,0.015941,0.793579,0.015461,...,-3.595513e-02,-4.319764e-02,2.874362e-02,-5.051845e-01,-2.852675e-01,0.015411,0.196400,-0.195925,1.0,1.0
24,Unilateral,US as Home,0.4,0.997303,0.138758,0.000036,0.4,0.015956,0.793579,0.015320,...,-3.750992e-02,-7.370434e-02,2.002651e-02,-6.739456e-01,-3.824218e-01,0.020443,0.348948,-0.348897,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1355,EPC_hybrid,US and EU as Home,45.0,0.893456,15.251869,0.841222,45.0,0.022703,0.792003,0.014242,...,-1.471185e+00,9.583382e-01,2.267209e+00,-6.887012e+00,-7.566471e+01,0.719148,442.204701,-34.166036,7.0,1.0
1361,EPC_hybrid,US and EU as Home,46.0,0.893162,15.585545,0.843979,46.0,0.022682,0.792003,0.014155,...,-1.475358e+00,9.644386e-01,2.275256e+00,-6.891185e+00,-7.733024e+01,0.723564,452.793723,-34.458061,7.0,1.0
1367,EPC_hybrid,US and EU as Home,47.0,0.892880,15.919194,0.846642,47.0,0.022661,0.792003,0.014070,...,-1.479368e+00,9.703074e-01,2.282994e+00,-6.895195e+00,-7.899573e+01,0.727843,463.389393,-34.744508,7.0,1.0
1373,EPC_hybrid,US and EU as Home,48.0,0.892609,16.252840,0.849216,48.0,0.022641,0.792003,0.013987,...,-1.483217e+00,9.759441e-01,2.290428e+00,-6.899044e+00,-8.066124e+01,0.731992,473.990895,-35.025622,7.0,1.0


In [28]:
Outcomes.to_csv('../output_theta{0}/output_case{1}_analytical.csv'.format(theta, case), header=True)

In [ ]:
temp_df = tax_scenario.apply(temp, axis=1, args=(alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit))
#temp_df.head()
output_list=[]
for i in range(2,len(tax_scenario)+2):
    output_list.append(temp_df.loc[i])
Outcomes = pd.concat(output_list, axis=0, join='outer', ignore_index=False, keys=tax_scenario['tax_sce'], levels=None, verify_integrity=False,copy=True)
Outcomes.reset_index(level=0, inplace=True)
Outcomes